# CulinaryFlo Datamart Integration with Everytable

By James, Curtis, and Richard begun November 2020

# Operational Queries

## Subscription Orders

In [4]:
-- SQL snippets from "ET-CF Test Data" documentation
-- https://docs.google.com/spreadsheets/d/1ooMnD9hrYZPGwEBwFJiyVWV1AE1206dYtyY_plA1B8E

SELECT 'Processing' AS 'Order Status'
       ,o.TotalAmount AS stripeFee
       ,CAST(o.Number AS nvarchar(99)) + '-' + CAST(p.Id AS nvarchar(99)) AS 'Order Number-Product Id'
       ,o.TotalAmount AS stripeNet
       ,o.CreatedDateUtc AS 'Order Date'
       ,o.Number AS 'Order Number'
       ,o.TotalAmount AS 'Order Total Amount'
       ,NULL AS Done -- Looks blank in Subs Orders
       ,o.CustomerId as 'Customer User ID'
       ,p.InternalName AS 'Item Name'
       ,i.TotalAmount AS 'Order Line Total (include tax)'
       ,i.OrderedQuantity AS 'Quantity (- Refund)' -- Or should it be (i.FulfilledQuantity - i.RefundedQuantity)
       ,p.Id AS 'Product Id'
       ,o.DeliveryZipCode AS 'Postcode (Shipping)'
       ,o.DeliveryFee AS 'Order Shipping Amount'
       ,CHOOSE(o.FulfillmentMethod + 1, 'Delivery Zone ' + dz.Name, s.Name) AS SHOP -- Repeated below
       ,prom.Name AS 'Coupon Code'
       ,NULL AS '#REF' -- Looks blank in Subs Orders
       ,o.DiscountAmount AS 'Discount Amount'
       ,NULL AS 'Is Renewal?' -- Likely not used per 11/20/20 emails
       ,o.FulfillmentDate AS dDate                                   -- Does it really needs conditions or is it fine? !!!!
       ,CHOOSE(o.FulfillmentMethod + 1,
          CAST(adzwi.StartHour AS nvarchar(9)) + ':00-' + CAST(adzwi.EndHour AS nvarchar(9)) + ':00', -- ScheduledDelivery, AM/PM?!!!
          CHOOSE(o.InStorePickupTime + 1, '9:01AM', '12:01PM', '3:01PM')) AS dTime -- InStorePickup window, wired as time for Subs Order
          /*CAST(o.OnDemandPickupDateUtc AT TIME ZONE 'UTC'
               AT TIME ZONE 'Pacific Standard Time' AS nvarchar(99))) AS dTime                        -- OndemandDelivery */
       ,c.PhoneNumber AS Phone
       ,c.Email AS Email
       ,c.FirstName + ' ' + c.LastName AS CustomerName
       ,o.Instructions AS 'Customer Note'
       ,CHOOSE(o.FulfillmentMethod + 1, 'delivery', 'pickup') AS Delivery_Type
       ,CHOOSE(o.FulfillmentMethod + 1, 'insulated_box', 'paper_bag') as packaging_type
       ,'_tbd-delivery' AS Store_ID                                          -- SHOULD BE home-delivery vs apartment-delivery !!!!
       ,CHOOSE(o.FulfillmentMethod + 1, 'Delivery Zone ' + dz.Name, s.Name) AS shipping_shop_pickup -- Repeated above
       ,o.Instructions AS 'Order Note'
       ,NULL AS '#N/A'       -- Let Subs Orders formual do it
       ,NULL AS 'a1, a2, a3' -- Let Subs Orders formula do it
       ,c.FirstName AS custFirst
       ,c.LastName AS custLast
       ,NULL AS ItemType  -- Looks blank in Subs Orders
       ,NULL AS MealCount -- Looks blank in Subs Orders
       ,o.DeliveryAddress1 AS a1
       ,o.DeliveryAddress2 AS a2
       ,o.DeliveryCity + ', ' + /*CAST(o.DeliveryState AS nvarchar(99))*/ 'CA' + ' ' + o.DeliveryZipCode AS a3
                                -- Returns code 8, hardwired to 'CA' for now
  FROM ActiveOrders AS o
  LEFT JOIN ActiveOrderItems AS i ON (o.Id = i.OrderId)
  LEFT JOIN Products AS p ON (i.ProductId = p.Id)
  LEFT JOIN Stores AS s ON (o.StoreId = s.Id)
  LEFT JOIN Customers AS c ON (o.CustomerId = c.Id)
  LEFT JOIN ActivePromotions AS prom ON (o.PromotionId = prom.Id)
  LEFT JOIN ActiveDeliveryZoneWindowInstances AS adzwi ON (o.DeliveryZoneWindowInstanceId = adzwi.Id)
  LEFT JOIN DeliveryZones AS dz ON (adzwi.DeliveryZoneId = dz.Id)
  WHERE o.FulfillmentMethod <= 1 -- Subscription Delivery and Subscription Store Pickup only, not On Demand

(259 rows affected)

Total execution time: 00:00:00.458

Order Status,stripeFee,Order Number-Product Id,stripeNet,Order Date,Order Number,Order Total Amount,Done,Customer User ID,Item Name,Order Line Total (include tax),Quantity (- Refund),Product Id,Postcode (Shipping),Order Shipping Amount,SHOP,Coupon Code,#REF,Discount Amount,Is Renewal?,dDate,dTime,Phone,Email,CustomerName,Customer Note,Delivery_Type,packaging_type,Store_ID,shipping_shop_pickup,Order Note,#N/A,"a1, a2, a3",custFirst,custLast,ItemType,MealCount,a1,a2,a3
Processing,21.48,267-764A18E0-88DF-42A9-9F5C-A02975A77827,21.48,2020-11-23 00:59:03.2683086,267,21.48,NULL,53cb6f9d-b50b-4e45-962c-c16e9bd4b619,Tangy BBQ Chicken Salad,7.16,1,764a18e0-88df-42a9-9f5c-a02975a77827,90007,0.00,Delivery Zone 2000,FALL20,NULL,5.37,NULL,2020-11-26 00:00:00.0000000,8:00-16:00,5413908660,joshua@vaneikeren.com,Josh van Eikeren,NULL,delivery,insulated_box,_tbd-delivery,Delivery Zone 2000,NULL,NULL,NULL,Josh,van Eikeren,NULL,NULL,dfsdsadsaf,NULL,"Los Angeles, CA 90007"
Processing,21.48,267-AC9C6CB7-52BA-4786-BF3C-D20D502FFB2A,21.48,2020-11-23 00:59:03.2683086,267,21.48,NULL,53cb6f9d-b50b-4e45-962c-c16e9bd4b619,Lemon Pepper Chicken Caesar Salad,7.16,1,ac9c6cb7-52ba-4786-bf3c-d20d502ffb2a,90007,0.00,Delivery Zone 2000,FALL20,NULL,5.37,NULL,2020-11-26 00:00:00.0000000,8:00-16:00,5413908660,joshua@vaneikeren.com,Josh van Eikeren,NULL,delivery,insulated_box,_tbd-delivery,Delivery Zone 2000,NULL,NULL,NULL,Josh,van Eikeren,NULL,NULL,dfsdsadsaf,NULL,"Los Angeles, CA 90007"
Processing,21.48,267-BAD2DB12-9F96-412B-9F81-7B65770A0516,21.48,2020-11-23 00:59:03.2683086,267,21.48,NULL,53cb6f9d-b50b-4e45-962c-c16e9bd4b619,BUFFALO CHICKEN SALAD,7.16,1,bad2db12-9f96-412b-9f81-7b65770a0516,90007,0.00,Delivery Zone 2000,FALL20,NULL,5.37,NULL,2020-11-26 00:00:00.0000000,8:00-16:00,5413908660,joshua@vaneikeren.com,Josh van Eikeren,NULL,delivery,insulated_box,_tbd-delivery,Delivery Zone 2000,NULL,NULL,NULL,Josh,van Eikeren,NULL,NULL,dfsdsadsaf,NULL,"Los Angeles, CA 90007"
Processing,52.56,261-0255A8C2-4FC3-4261-9C40-71B670DC132F,52.56,2020-11-22 19:25:32.5830840,261,52.56,NULL,53cb6f9d-b50b-4e45-962c-c16e9bd4b619,Homegirl Salmon Adobo,8.76,1,0255a8c2-4fc3-4261-9c40-71b670dc132f,NULL,0.00,Delivery Zone 2000,FALL20,NULL,13.14,NULL,2020-11-26 00:00:00.0000000,8:00-16:00,5413908660,joshua@vaneikeren.com,Josh van Eikeren,hkjhkjkhj,delivery,insulated_box,_tbd-delivery,Delivery Zone 2000,hkjhkjkhj,NULL,NULL,Josh,van Eikeren,NULL,NULL,NULL,NULL,NULL
Processing,52.56,261-EB8B6448-C848-41A7-81BC-99E54D2512B3,52.56,2020-11-22 19:25:32.5830840,261,52.56,NULL,53cb6f9d-b50b-4e45-962c-c16e9bd4b619,Balsamic Chicken Grain Bowl,17.52,2,eb8b6448-c848-41a7-81bc-99e54d2512b3,NULL,0.00,Delivery Zone 2000,FALL20,NULL,13.14,NULL,2020-11-26 00:00:00.0000000,8:00-16:00,5413908660,joshua@vaneikeren.com,Josh van Eikeren,hkjhkjkhj,delivery,insulated_box,_tbd-delivery,Delivery Zone 2000,hkjhkjkhj,NULL,NULL,Josh,van Eikeren,NULL,NULL,NULL,NULL,NULL
Processing,52.56,261-C8E26B0D-0C1E-42A5-9792-3D66A2015EAD,52.56,2020-11-22 19:25:32.5830840,261,52.56,NULL,53cb6f9d-b50b-4e45-962c-c16e9bd4b619,Butter Chicken,26.28,3,c8e26b0d-0c1e-42a5-9792-3d66a2015ead,NULL,0.00,Delivery Zone 2000,FALL20,NULL,13.14,NULL,2020-11-26 00:00:00.0000000,8:00-16:00,5413908660,joshua@vaneikeren.com,Josh van Eikeren,hkjhkjkhj,delivery,insulated_box,_tbd-delivery,Delivery Zone 2000,hkjhkjkhj,NULL,NULL,Josh,van Eikeren,NULL,NULL,NULL,NULL,NULL
Processing,17.52,277-30F73BB9-9DBE-4B8F-A4F0-F1DAAC331790,17.52,2020-11-23 23:43:35.8562768,277,17.52,NULL,c1d9307c-66b4-417f-b49c-e7ebdd77f116,Turkey Taco Bowl,8.76,1,30f73bb9-9dbe-4b8f-a4f0-f1daac331790,90056,0.00,Delivery Zone 2000,FALL20,NULL,4.38,NULL,2020-11-26 00:00:00.0000000,8:00-16:00,5129543328,meganmclindon+testCF2@gmail.com,Megan M.,NULL,delivery,insulated_box,_tbd-delivery,Delivery Zone 2000,NULL,NULL,NULL,Megan,M.,NULL,NULL,6022 Flores Ave,NULL,"Los Angeles, CA 90056"
Processing,17.52,277-CD3B9072-9B21-4113-9B3F-8A1AA230EB6B,17.52,2020-11-23 23:43:35.8562768,277,17.52,NULL

## Store Orders

In [5]:
-- Store orders aka ActiveStoreProductInventoryRequests
SELECT o.Date
      ,s.Name as Store
      ,pc.Name as Category
      ,p.InternalName as Item
      ,SUM(o.ProductCount) as Quantity
  FROM ActiveStoreProductInventoryRequests AS o
  LEFT JOIN Products as p ON (o.ProductId = p.Id)
  LEFT JOIN Stores as s ON (o.StoreId = s.Id)
  LEFT JOIN ProductCategories as pc ON (p.CategoryId = pc.Id)
  GROUP BY o.date, s.Name, pc.Name, p.InternalName
  ORDER BY o.date, s.Name, pc.Name, p.InternalName

Warning: Null value is eliminated by an aggregate or other SET operation.

(1599 rows affected)

Total execution time: 00:00:00.392

Date,Store,Category,Item,Quantity
2020-11-23 00:00:00.0000000,BLOC - DTLA,Breakfast,Monica's Breakfast Burrito,13
2020-11-23 00:00:00.0000000,BLOC - DTLA,Breakfast,OATS WITH FRUIT & NUTS,9
2020-11-23 00:00:00.0000000,BLOC - DTLA,Breakfast,Scrambled Eggs & Veggies,9
2020-11-23 00:00:00.0000000,BLOC - DTLA,Breakfast,Southwest Breakfast Bowl,9
2020-11-23 00:00:00.0000000,BLOC - DTLA,Breakfast,Turkey Sausage & Egg White Wrap,9
2020-11-23 00:00:00.0000000,BLOC - DTLA,Hot Plates,Balsamic Chicken Grain Bowl,19
2020-11-23 00:00:00.0000000,BLOC - DTLA,Hot Plates,Butter Chicken,17
2020-11-23 00:00:00.0000000,BLOC - DTLA,Hot Plates,Carnitas Bowl,19
2020-11-23 00:00:00.0000000,BLOC - DTLA,Hot Plates,Chimichurri Chicken,10
2020-11-23 00:00:00.0000000,BLOC - DTLA,Hot Plates,Golden Couscous Bowl,12


# R365 Upload Queries<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif;"></span>

## 1\. R365-JounalEntry

TBD

## 2\. R365-Sales

In [6]:
-- R365-Sales
-- per https://docs.google.com/spreadsheets/d/1ooMnD9hrYZPGwEBwFJiyVWV1AE1206dYtyY_plA1B8E/edit?pli=1#gid=486805977
-- Receipt Number, Date, Service Type, Server, Tax Amount, Tip Amount, Number of Guests

SELECT o.Number AS 'Receipt Number'
      ,o.CreatedDateUtc AT TIME ZONE 'UTC' AT TIME ZONE 'Pacific Standard Time' AS 'Date'
      ,CHOOSE(o.FulfillmentMethod + 1, 'ScheduledDelivery', 'InStorePickup', 'OndemandDelivery',
                                       'InStoreDining', 'InStoreToGo', 'In-Store (Other)') AS 'Service Type'
      ,u.FirstName +' '+ u.LastName AS 'Server'
      ,o.TaxAmount AS 'Tax Amount'
      ,NULL AS 'Tip Amount'
      ,NULL AS 'Number of Guests'
  FROM (SELECT * FROM ActiveOrders UNION ALL SELECT * FROM ClosedOrders) AS o
  LEFT JOIN UserProfiles AS u ON (o.UserId = u.Id)
  ORDER BY o.Number

(296 rows affected)

Total execution time: 00:00:01.051

Receipt Number,Date,Service Type,Server,Tax Amount,Tip Amount,Number of Guests
4,2020-10-31 12:53:54.7137641 -07:00,OndemandDelivery,NULL,0.00,NULL,NULL
5,2020-10-31 12:54:52.0413540 -07:00,OndemandDelivery,NULL,0.00,NULL,NULL
6,2020-10-31 12:59:39.8949026 -07:00,OndemandDelivery,NULL,0.00,NULL,NULL
7,2020-10-31 13:05:48.8908704 -07:00,InStorePickup,NULL,0.00,NULL,NULL
8,2020-10-31 14:06:06.4311962 -07:00,InStorePickup,NULL,0.00,NULL,NULL
9,2020-10-31 14:07:18.5519607 -07:00,InStorePickup,NULL,0.00,NULL,NULL
10,2020-10-31 20:04:03.2377557 -07:00,InStorePickup,NULL,0.00,NULL,NULL
11,2020-10-31 20:05:46.2758959 -07:00,InStorePickup,NULL,0.00,NULL,NULL
12,2020-11-01 16:53:28.0946841 -08:00,InStorePickup,NULL,0.00,NULL,NULL
13,2020-11-01 16:54:36.8340655 -08:00,ScheduledDelivery,NULL,0.00,NULL,NULL


## 3\. R365-Sales Payments

In [7]:
-- R365-Sales Payments
-- per https://docs.google.com/spreadsheets/d/1ooMnD9hrYZPGwEBwFJiyVWV1AE1206dYtyY_plA1B8E/edit?pli=1#gid=1114661678
-- Receipt Number, Date, Payment Type or Payment Type, Amount, Credit Card Fee (optional)

SELECT o.Number AS 'Receipt Number'
      ,o.CreatedDateUtc AT TIME ZONE 'UTC' AT TIME ZONE 'Pacific Standard Time' AS 'Date'
      ,IIF(o.PromotionID IS NULL,
           CHOOSE(pay.Tender + 1, 'Card', 'Cash', 'Account Credit', 'Loyalty', 'EBT', 'Other', 'Gift Card'),
           pro.Code) AS 'Payment Type'
      ,IIF(o.PromotionID IS NULL, o.TotalAmount, o.DiscountAmount) AS 'Amount' -- DOUBLE COUNTING BECAUSE OF JOIN? !!!!
      ,NULL AS 'Credit Card Fee (optional)'
  FROM (SELECT * FROM ActiveOrders UNION ALL SELECT * FROM ClosedOrders) AS o
  LEFT JOIN Payments AS pay ON (o.Id = pay.OrderId)
  LEFT JOIN ActivePromotions AS pro ON (o.PromotionId = pro.Id)
  ORDER BY o.Number

(343 rows affected)

Total execution time: 00:00:00.127

Receipt Number,Date,Payment Type,Amount,Credit Card Fee (optional)
4,2020-10-31 12:53:54.7137641 -07:00,Card,32.52,NULL
5,2020-10-31 12:54:52.0413540 -07:00,Card,32.52,NULL
6,2020-10-31 12:59:39.8949026 -07:00,Card,36.02,NULL
7,2020-10-31 13:05:48.8908704 -07:00,Card,24.80,NULL
7,2020-10-31 13:05:48.8908704 -07:00,Loyalty,24.80,NULL
8,2020-10-31 14:06:06.4311962 -07:00,Card,17.90,NULL
9,2020-10-31 14:07:18.5519607 -07:00,Card,32.80,NULL
10,2020-10-31 20:04:03.2377557 -07:00,Card,150.30,NULL
11,2020-10-31 20:05:46.2758959 -07:00,Loyalty,157.25,NULL
11,2020-10-31 20:05:46.2758959 -07:00,Card,157.25,NULL


## 4\. R365-Sales Details

In [8]:
-- R365-Sales Details
-- per https://docs.google.com/spreadsheets/d/1ooMnD9hrYZPGwEBwFJiyVWV1AE1206dYtyY_plA1B8E/edit?pli=1#gid=127241649
-- Receipt Number, Menu Item, Quantity, Amount, Category, Void

SELECT o.Number AS 'Receipt Number'
      ,prod.InternalName AS 'Menu Item'
      ,i.FulfilledQuantity - i.RefundedQuantity AS 'Quantity' -- Or OrderedQuantity?
      ,i.TotalAmount AS 'Amount'
      ,cat.Name AS 'Category'
      ,NULL AS 'Void'
  FROM (SELECT * FROM ActiveOrders UNION ALL SELECT * FROM ClosedOrders) AS o
  LEFT JOIN (SELECT * FROM ActiveOrderItems UNION ALL SELECT * FROM ClosedOrderItems) AS i ON (o.Id = i.OrderId)
  LEFT JOIN Products AS prod ON (i.ProductId = prod.Id)
  LEFT JOIN ProductCategories AS cat ON (prod.CategoryId = cat.Id)
  ORDER BY o.Number

(1251 rows affected)

Total execution time: 00:00:00.267

Receipt Number,Menu Item,Quantity,Amount,Category,Void
4,BUFFALO CHICKEN SALAD,0,6.95,Salads,NULL
4,Salmon Superfood Salad,0,6.95,Salads,NULL
4,Lemon Pepper Chicken Caesar Salad,0,6.95,Salads,NULL
5,Lemon Pepper Chicken Caesar Salad,0,6.95,Salads,NULL
5,Salmon Superfood Salad,0,6.95,Salads,NULL
5,BUFFALO CHICKEN SALAD,0,6.95,Salads,NULL
6,BUFFALO CHICKEN SALAD,0,6.95,Salads,NULL
6,Salmon Superfood Salad,0,6.95,Salads,NULL
6,Vegan Rainbow Salad,0,3.95,Salads,NULL
6,Tangy BBQ Chicken Salad,0,6.95,Salads,NULL


## 5\. R365-Drawer

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif;">TBD</span>

# Misc Queries

In [9]:
SELECT * FROM sys.time_zone_info

(140 rows affected)

Total execution time: 00:00:00.054

name,current_utc_offset,is_currently_dst
Dateline Standard Time,-12:00,0
UTC-11,-11:00,0
Aleutian Standard Time,-10:00,0
Hawaiian Standard Time,-10:00,0
Marquesas Standard Time,-09:30,0
Alaskan Standard Time,-09:00,0
UTC-09,-09:00,0
Yukon Standard Time,-07:00,1
Pacific Standard Time (Mexico),-08:00,0
UTC-08,-08:00,0
